GROUP 54 SHADOW MODELS

In [1]:
# =========
# installs
# =========
!pip install transformers
!pip install datasets
!pip install scikit-learn
!pip install seaborn
!pip install tf_keras

In [3]:
# ========
# Imports
# ========
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models    import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers    import Input, Dense, LeakyReLU, Concatenate
from tensorflow.keras import regularizers
from sklearn.metrics            import accuracy_score, f1_score, classification_report
from tensorflow.keras.layers import Dropout
from sklearn.metrics import precision_recall_fscore_support

In [5]:
# ===========================
# Step 1: Loading Embeddings
# ===========================
x1_shadow_train = np.load("x1_shadow_train.npy")
x2_shadow_train = np.load("x2_shadow_train.npy")
y_shadow_train  = np.load("y_shadow_train.npy")
x1_shadow_test  = np.load("x1_shadow_test.npy")
x2_shadow_test  = np.load("x2_shadow_test.npy")
y_shadow_test   = np.load("y_shadow_test.npy")

print("x1_shadow_train shape:", x1_shadow_train.shape)
print("x2_shadow_train shape:", x2_shadow_train.shape)
print("y_shadow_train shape:", y_shadow_train.shape)
print("x1_shadow_test shape:", x1_shadow_test.shape)
print("x2_shadow_test shape:", x2_shadow_test.shape)
print("y_shadow_test shape:", y_shadow_test.shape)

x1_shadow_train shape: (32000, 768)
x2_shadow_train shape: (32000, 768)
y_shadow_train shape: (32000,)
x1_shadow_test shape: (8000, 768)
x2_shadow_test shape: (8000, 768)
y_shadow_test shape: (8000,)


In [7]:
# ==================================
# Step 3: Build Siamese Autoencoder
# ==================================
def build_siamese_autoencoder(embedding_dim):
    encoder_input = Input(shape=(embedding_dim,))
    x = layers.Dense(50, activity_regularizer=regularizers.l1(0.01))(encoder_input)
    x = layers.LeakyReLU(alpha=0.01)(x)
    encoder_output = layers.Dense(embedding_dim, activation='relu')(x)
    encoder = Model(encoder_input, encoder_output)

    decoder_input = Input(shape=(embedding_dim,))
    decoder_output = layers.Dense(embedding_dim, activation='sigmoid')(decoder_input)
    decoder = Model(decoder_input, decoder_output)

    input1 = Input(shape=(embedding_dim,))
    input2 = Input(shape=(embedding_dim,))
    encoded1 = encoder(input1)
    encoded2 = encoder(input2)
    recon1 = decoder(encoded1)
    recon2 = decoder(encoded2)

    merged_output = layers.Concatenate()([recon1, recon2])
    model = Model(inputs=[input1, input2], outputs=merged_output)
    return model, encoder

def hybrid_classification_loss(margin=2.5, alpha=1.0):
    def loss_fn(y_true, y_pred):
        emb_dim = tf.shape(y_pred)[1] // 2
        recon1 = y_pred[:, :emb_dim]
        recon2 = y_pred[:, emb_dim:]
        recon_loss = tf.reduce_mean(tf.square(recon1 - recon2), axis=1)
        distances = tf.sqrt(tf.reduce_sum(tf.square(recon1 - recon2), axis=1))
        y_true = tf.cast(y_true, tf.float32)
        contrastive_loss = y_true * tf.square(distances) + (1 - y_true) * tf.square(tf.maximum(margin - distances, 0))
        return tf.reduce_mean(alpha * recon_loss + contrastive_loss)
    return loss_fn
embedding_dim = x1_shadow_train.shape[1]

In [9]:
# ===================================================================
# Step 4: Split Train & Test Embeddings into Number of Shadow Models
# ===================================================================
numShadowModels = 3;

n_train = len(y_shadow_train)
idx_splits_train = np.array_split(np.arange(n_train), numShadowModels)

n_test = len(y_shadow_test)
idx_splits_test = np.array_split(np.arange(n_test), numShadowModels)

In [11]:
# ==============================================
# Step 5: Train Three Independent Shadow Models
# ==============================================
shadow_models = []
shadow_mlps = []
shadow_encoders = []

for i in range(numShadowModels):
    train_idxs = idx_splits_train[i]
    x1_i = x1_shadow_train[train_idxs]
    x2_i = x2_shadow_train[train_idxs]
    y_i  = y_shadow_train[train_idxs]

    model_i, encoder_i = build_siamese_autoencoder(embedding_dim)
    model_i.compile(
        optimizer='adam',
        loss=hybrid_classification_loss(margin=2.5, alpha=1.0)
    )

    print(f"\n--- Training shadow model #{i+1} on {len(train_idxs)} examples ---")
    model_i.fit(
        [x1_i, x2_i], y_i,
        epochs=30, batch_size=256,
        validation_split=0.1,
        verbose=1
    )

    shadow_models.append(model_i)
    shadow_encoders.append(encoder_i)

    encoder_i.save(f"shadow_encoder_{i+1}.h5")



--- Training shadow model #1 on 10667 examples ---
Epoch 1/30


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 66.9716 - val_loss: 24.6692
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23.3380 - val_loss: 18.6405
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 18.4217 - val_loss: 16.2679
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 16.0546 - val_loss: 14.8947
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 14.5832 - val_loss: 13.6938
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 13.5336 - val_loss: 13.0284
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12.7601 - val_loss: 12.2633
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 12.0031 - val_loss: 11.7428
Epoch 9/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 11.4527 - val_loss: 11.4217
Epoch 10/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.9567 - val_loss: 10.9169
Epoch 11/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.4930 - val_loss: 10.5941
Epoch 12/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 


--- Training shadow model #2 on 10667 examples ---
Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 61.0257 - val_loss: 23.8255
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 22.3845 - val_loss: 18.1925
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 17.8508 - val_loss: 15.8599
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 15.6411 - val_loss: 14.4205
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 14.2361 - val_loss: 13.3220
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 13.1565 - val_loss: 12.4605
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 12.3965 - val_loss: 11.8270
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 11.6403 - val_loss: 11.4216
Epoch 9/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 11.2156 - val_loss: 10.9482
Epoch 10/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.7717 - val_loss: 10.7801
Epoch 11/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.4003 - val_loss:


--- Training shadow model #3 on 10666 examples ---
Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 67.7522 - val_loss: 24.4581
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 22.8117 - val_loss: 18.5328
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 18.1141 - val_loss: 16.0258
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 15.9715 - val_loss: 14.7507
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 14.4819 - val_loss: 13.5110
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 13.3473 - val_loss: 12.7058
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 12.5481 - val_loss: 12.0264
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 11.9352 - val_loss: 11.5286
Epoch 9/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 11.2536 - val_loss: 11.2093
Epoch 10/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 10.8482 - val_loss: 10.6923
Epoch 11/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.4409 - val_loss:

In [13]:
# ============================================================
# Step 6: Evaluate & Add Classification Heads for Each Shadow
# ============================================================
for i in range(numShadowModels):
    print(f"\n=== Evaluating Shadow Model #{i+1} ===")

    # grab this shadow’s encoder
    encoder_i = shadow_encoders[i]

    # select this model's test indices
    test_idxs = idx_splits_test[i]
    x1_test_i = x1_shadow_test[test_idxs]
    x2_test_i = x2_shadow_test[test_idxs]
    y_test_i  = y_shadow_test[test_idxs]

    # 1) Distance‐based evaluation on split test
    encoded1_test = encoder_i.predict(x1_test_i)
    encoded2_test = encoder_i.predict(x2_test_i)
    test_distances = np.linalg.norm(encoded1_test - encoded2_test, axis=1)

    threshold = 1.0
    test_pred = (test_distances < threshold).astype(int)

    acc_dist = accuracy_score(y_test_i, test_pred)
    f1_dist  = f1_score(y_test_i, test_pred)
    print(f"Distance‐based → Acc: {acc_dist:.4f}, F1: {f1_dist:.4f}")

    # 2) Siamese‐style MLP classification on this split
    train_idxs = idx_splits_train[i]
    encoded1_tr = encoder_i.predict(x1_shadow_train[train_idxs])
    encoded2_tr = encoder_i.predict(x2_shadow_train[train_idxs])
    diff_train  = np.abs(encoded1_tr - encoded2_tr)
    diff_test   = np.abs(encoded1_test - encoded2_test)

    input_diff = Input(shape=(diff_train.shape[1],))
    x = Dense(64, activation='relu')(input_diff)
    x = Dense(32, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    clf_model = Model(inputs=input_diff, outputs=output)
    clf_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    clf_model.fit(
        diff_train, y_shadow_train[train_idxs],
        epochs=20, batch_size=256,
        validation_split=0.1,
        verbose=0
    )

    y_prob = clf_model.predict(diff_test).flatten()
    y_pred = (y_prob > 0.5).astype(int)

    acc_clf = accuracy_score(y_test_i, y_pred)
    f1_clf  = f1_score(y_test_i, y_pred)
    print(f"Classifier @0.5 → Acc: {acc_clf:.4f}, F1: {f1_clf:.4f}")
    print(classification_report(y_test_i, y_pred))

    # 3) Tune threshold on pseudo‐val split
    diff_full = diff_train
    y_full    = y_shadow_train[train_idxs]
    d_tr, d_val, y_tr, y_val = train_test_split(
        diff_full, y_full,
        test_size=0.2, random_state=42,
        stratify=y_full
    )

    input_d2 = Input(shape=(d_tr.shape[1],))
    y2 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_d2)
    y2 = Dropout(0.5)(y2)
    y2 = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(y2)
    out2 = Dense(1, activation='sigmoid')(y2)
    clf2 = Model(inputs=input_d2, outputs=out2)
    clf2.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    clf2.fit(d_tr, y_tr, epochs=30, batch_size=256, validation_split=0.1, verbose=0)

    y_val_prob = clf2.predict(d_val).flatten()
    best_thr, best_f1 = 0.5, 0.0
    for thr in np.arange(0.1, 0.9, 0.01):
        yv = (y_val_prob >= thr).astype(int)
        _, _, f1s, _ = precision_recall_fscore_support(y_val, yv, average='binary')
        if f1s > best_f1:
            best_f1, best_thr = f1s, thr

    print(f"Tuned threshold on pseudo‐val → {best_thr:.2f} (F1={best_f1:.4f})")

    y_test_prob = clf2.predict(diff_test).flatten()
    y_test_pred = (y_test_prob >= best_thr).astype(int)
    acc_final = accuracy_score(y_test_i, y_test_pred)
    f1_final  = f1_score(y_test_i, y_test_pred)
    print(f"Final eval @thr={best_thr:.2f} → Acc: {acc_final:.4f}, F1: {f1_final:.4f}")

    clf2.save(f"shadow_mlp_{i+1}.h5")
    np.save(f"shadow_threshold_{i+1}.npy", np.array([best_thr]))

    # Save predictions for this shadow's test split
    pd.DataFrame({
        "True Label": y_test_i,
        "Predicted Label": y_test_pred,
        "Confidence": y_test_prob
    }).to_csv(f"shadow{i+1}_test_predictions.csv", index=False)


=== Evaluating Shadow Model #1 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step
Distance‐based → Acc: 0.5028, F1: 0.6683
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 217us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step
Classifier @0.5 → Acc: 0.7904, F1: 0.7870
              precision    recall  f1-score   support

         0.0       0.78      0.81      0.79      1331
         1.0       0.80      0.77      0.79      1336

    accuracy                           0.79      2667
   macro avg       0.79      0.79      0.79      2667
weighted avg       0.79      0.79      0.79      2667

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step
Tuned threshold on pseudo‐val → 0.51 (F1=0.8518)
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step


Final eval @thr=0.51 → Acc: 0.7968, F1: 0.8012

=== Evaluating Shadow Model #2 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 271us/step
Distance‐based → Acc: 0.4949, F1: 0.6622
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step
Classifier @0.5 → Acc: 0.7777, F1: 0.7635
              precision    recall  f1-score   support

         0.0       0.75      0.83      0.79      1347
         1.0       0.81      0.72      0.76      1320

    accuracy                           0.78      2667
   macro avg       0.78      0.78      0.78      2667
weighted avg       0.78      0.78      0.78      2667

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step
Tuned threshold on pseudo‐val → 0.50 (F1=0.8552)
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 235us/step


Final eval @thr=0.50 → Acc: 0.7683, F1: 0.7758

=== Evaluating Shadow Model #3 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step
Distance‐based → Acc: 0.5098, F1: 0.6741
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 219us/step
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step
Classifier @0.5 → Acc: 0.7787, F1: 0.7823
              precision    recall  f1-score   support

         0.0       0.78      0.77      0.77      1314
         1.0       0.78      0.78      0.78      1352

    accuracy                           0.78      2666
   macro avg       0.78      0.78      0.78      2666
weighted avg       0.78      0.78      0.78      2666

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step
Tuned threshold on pseudo‐val → 0.44 (F1=0.8652)
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step


Final eval @thr=0.44 → Acc: 0.7761, F1: 0.7918
